# Building blocks in Haystack: components and pipelines

In the [previous notebook](data_classes.ipynb), we learned how we can store structured and unstructured data through Documents objects, as well as dataframe, ByteStream, ChatMessage and StreamingChunk objects. We also learned how to store these objects into a Document Store. In this notebook, we will explore how to store and retrieve data from a Haystack Document store. Let's take a look at its architecture.

Haystack's architecture leverages components as its core elements, each performing specific functions like text processing or summarization. These components are designed to be connected into pipelines, which orchestrate the flow of data and manage task execution in a structured manner. The Pipeline class facilitates this by allowing the addition and connection of components, which must have unique input and output points for data transfer.

Pipelines are the backbone of NLP applications in Haystack, functioning as directed graphs where nodes are components and edges dictate data flow. They ensure smooth data processing, handle errors, and support debugging through visualization tools that help developers trace and optimize the data journey.

Haystack emphasizes modularity and flexibility, providing a range of pre-built components while also supporting custom ones for specific needs. The framework's pipelines enable the assembly of sophisticated NLP applications, integrating various functionalities into a cohesive system. In this notebook we will explore key components. In  the pipelines.ipynb notebook we will see how to connect them into pipelines.

### Introduction to components

Within Haystack, we can find the following key ready-made components. There are more, but for now we will focus on these as we get started with Haystack's functionality.

![](./images/haystack-components.png)

### Embedding components

In this example, `docs` is a list of `Document` objects with text content to be embedded. The `OpenAIDocumentEmbedder` is initialized with an OpenAI API key and is used to generate embeddings for each document. The embeddings are then printed out for each document in the `docs` list.

#### OpenAIDocumentEmbedder

In [8]:
from haystack.preview import Document
from haystack.preview.components.embedders import OpenAIDocumentEmbedder
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv("./../../.env")
api_key = os.getenv("OPENAI_API_KEY")

# List of documents to embed
docs = [Document(content="The quick brown fox jumps over the lazy dog."), 
        Document(content="To be or not to be, that is the question.")]

# Initialize the embedder with your OpenAI API key
document_embedder = OpenAIDocumentEmbedder(api_key=api_key)

# Run the embedder to get embeddings
result = document_embedder.run(docs)

# Access the embeddings stored in the documents
for doc in result['documents']:
    print(doc.embedding)

Calculating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Calculating embeddings: 100%|██████████| 1/1 [00:00<00:00,  3.18it/s]

[0.0016260817646980286, 0.005972211714833975, -0.014899228699505329, -0.008626528084278107, -0.011499950662255287, 0.015537766739726067, -0.021434856578707695, -0.008682869374752045, -0.010905234143137932, -0.022449005395174026, 0.02130965329706669, 0.017703790217638016, 0.02211095578968525, -0.0026777919847518206, 0.0007449614349752665, -0.012113448232412338, 0.026142511516809464, -0.008632788434624672, 0.01363467238843441, -0.012432717718183994, -0.013446866534650326, 0.023200226947665215, -0.0051646484062075615, -0.005230380687862635, 0.007975469343364239, 0.021046726033091545, 0.010254175402224064, 0.0007926953257992864, 0.0027904752641916275, -0.005781276151537895, 0.015763133764266968, -0.0021879328414797783, -0.03615880012512207, -0.0011683061020448804, -0.010967835783958435, -0.014122966676950455, 0.0020392537117004395, -0.006523107644170523, 0.024226896464824677, -0.04657573997974396, 0.011506211012601852, 0.021647702902555466, -0.025604136288166046, -0.005872048903256655, 0.0

Taking a look at the embeddings

In [15]:
result['documents'][0].embedding

[0.0016260817646980286,
 0.005972211714833975,
 -0.014899228699505329,
 -0.008626528084278107,
 -0.011499950662255287,
 0.015537766739726067,
 -0.021434856578707695,
 -0.008682869374752045,
 -0.010905234143137932,
 -0.022449005395174026,
 0.02130965329706669,
 0.017703790217638016,
 0.02211095578968525,
 -0.0026777919847518206,
 0.0007449614349752665,
 -0.012113448232412338,
 0.026142511516809464,
 -0.008632788434624672,
 0.01363467238843441,
 -0.012432717718183994,
 -0.013446866534650326,
 0.023200226947665215,
 -0.0051646484062075615,
 -0.005230380687862635,
 0.007975469343364239,
 0.021046726033091545,
 0.010254175402224064,
 0.0007926953257992864,
 0.0027904752641916275,
 -0.005781276151537895,
 0.015763133764266968,
 -0.0021879328414797783,
 -0.03615880012512207,
 -0.0011683061020448804,
 -0.010967835783958435,
 -0.014122966676950455,
 0.0020392537117004395,
 -0.006523107644170523,
 0.024226896464824677,
 -0.04657573997974396,
 0.011506211012601852,
 0.021647702902555466,
 -0.0256

#### OpenAITextEmbedder

In this snippet, `text_embedder` is created with an OpenAI API key and used to generate an embedding for the string "I love pizza!". The resulting embedding and associated metadata are then printed out.

In [5]:
from haystack.preview.components.embedders import OpenAITextEmbedder

# Initialize the text embedder with your OpenAI API key
text_embedder = OpenAITextEmbedder(api_key=api_key)

# Text you want to embed
text_to_embed = "I love pizza!"

# Embed the text and print the result
result = text_embedder.run(text_to_embed)
print(result)

# Output will include the embedding vector and metadata
# {'embedding': [0.017020374536514282, -0.023255806416273117, ...],
#  'metadata': {'model': 'text-embedding-ada-002',
#               'usage': {'prompt_tokens': 4, 'total_tokens': 4}}}


{'embedding': [0.017020374536514282, -0.023255806416273117, 0.007667226251214743, -0.033693648874759674, -0.029528429731726646, -0.011646497994661331, -0.006254027131944895, -0.009080425836145878, 0.026429306715726852, -0.03461098670959473, 0.030222631990909576, 0.009644465520977974, -0.00958248320966959, -0.00684905843809247, -0.001529416418634355, -0.011026673950254917, 0.030445769429206848, -0.0059379166923463345, 0.010679572820663452, -0.02620617114007473, -0.012904741801321507, -0.014565871097147465, 0.004394554067403078, -0.020726924762129784, -0.008547376841306686, 0.02843753807246685, -0.0006543020135723054, -0.0023971705231815577, 0.008144491352140903, -0.010425444692373276, 0.04705705866217613, 0.0069234371185302734, -0.02453264594078064, -0.01952446438372135, -0.010419245809316635, -0.005048468708992004, -0.00459909625351429, -0.010208506137132645, -0.009185795672237873, -0.012557639740407467, 0.010196109302341938, -0.012941931374371052, -0.009886197745800018, -0.01316506788